<a href="https://colab.research.google.com/github/benmsanderson/prpattern/blob/main/CMIP6_spatial_emulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
datadir='/content/drive/MyDrive/colab_4xco2'

In [2]:
!pip install eofs
!pip install lmfit corner emcee netcdf4 h5netcdf


In [3]:
#from matplotlib import pyplot as plt
import pickle
import os
import scipy
#import importlib

from scipy import signal
import numpy as np
import emcee
import lmfit
import pandas as pd
import xarray as xr
import sys
from eofs.xarray import Eof



In [4]:
sys.path.append(os.path.join(os.getcwd(), "../", "src"))

In [5]:
from meteor.utils import make_anom

Open xarray datasets

In [6]:
mdlslr=pickle.load( open( datadir+"/mdls.pkl", "rb" ))
nmlr=len(mdlslr)

In [7]:
datadir+"/abrupt-4xCO2.nc"

'/content/drive/MyDrive/colab_4xco2/abrupt-4xCO2.nc'

In [8]:
#CMIP6
ds_4x = xr.open_dataset(datadir+"/abrupt-4xCO2.nc")
ds_1pc = xr.open_dataset(datadir+"/1pctCO2.nc")
ds_hist = xr.open_dataset(datadir+"/historical.nc")

ds_cnt = xr.open_dataset(datadir+"/piControl.nc")
ds_r85 = xr.open_dataset(datadir+"/ssp585.nc")
ds_r26 = xr.open_dataset(datadir+"/ssp126.nc")

In [ ]:
if 1:
  ds_4x_anom=make_anom(ds_4x,ds_cnt)
  ds_4x_anom.to_netcdf(datadir+'/a4xanom.nc')
  ds_1pc_anom=make_anom(ds_1pc,ds_cnt)
  ds_1pc_anom.to_netcdf(datadir+'/1pcanom.nc')

else:
  ds_4x_anom=xr.open_dataset('/content/drive/MyDrive/colab_4xco2/a4xanom.nc')
  ds_1pc_anom=xr.open_dataset('/content/drive/MyDrive/colab_4xco2/1pcanom.nc')

In [ ]:
X=ds_4x_anom.tas[0,:150,:,:]


In [ ]:
(ts,out,us,orgeof)=prpatt.get_timescales(X,[5,40,10000])
ts

In [ ]:
np.tile(orgeof['s'].T,(150,1)).shape
orgeof['u'].shape

In [ ]:
p,ax=plt.subplots(3,1)
p.set_size_inches(4,10)

orgeof['v'][0,:,:].plot(ax=ax[0])
orgeof['v'][1,:,:].plot(ax=ax[1])
orgeof['v'][2,:,:].plot(ax=ax[2])


In [ ]:
plt.plot(us)
plt.plot(orgeof['u'])
orgeof['u'].shape[0]

In [ ]:
Xrp=prpatt.recon(orgeof)
Xrs=prpatt.rmodel(orgeof,us)



In [ ]:
X.weighted(prpatt.wgt(X)).mean(('lat','lon')).plot()
Xrp.weighted(prpatt.wgt(X)).mean(('lat','lon')).plot()
Xrs.weighted(prpatt.wgt(X)).mean(('lat','lon')).plot()



In [ ]:
p,ax=plt.subplots(2,1)
p.set_size_inches(4,8)
Xrs[-1,:,:].plot(ax=ax[0])
ax[0].set_title('emulated year 150 abrupt4x')
X[-1,:,:].plot(ax=ax[1])
ax[1].set_title('actual year 150 abrupt4x')


In [ ]:
f=np.arange(0,150)*5.35*np.log(1.01)
Xu=prpatt.imodel_eof(out.params, f)
Xf=prpatt.imodel_filter(out.params,f)
Xsim=prpatt.rmodel(orgeof,Xf)

In [ ]:
plt.plot(f)
plt.ylabel(r'Wm$^{-2}$')
plt.xlabel('year')
plt.title('1pctCO2 forcing timeseries')

In [ ]:
plt.plot(Xf[:])
#plt.plot(Xu[:],'--')

In [ ]:
p1=Xsim.weighted(prpatt.wgt(X)).mean(('lat','lon')).plot()
Xact=ds_1pc_anom.tas[0,:150,:,:]

p2=Xact.weighted(prpatt.wgt(X)).mean(('lat','lon')).plot()
plt.ylabel(r'Wm$^{-2}$')
plt.xlabel('year')
plt.title('1pctCO2 global mean temperature response')
plt.legend((p1[0],p2[0]),['Emulated','truth'])

In [ ]:
p,ax=plt.subplots(2,1)
p.set_size_inches(8,8)
Xsim[-1,:,:].plot(ax=ax[0])
ax[0].set_title('emulated year 150 1pctCO2')
Xact[-1,:,:].plot(ax=ax[1])
ax[1].set_title('actual year 150 1pctCO2')


In [ ]:
solver = Eof(X,center=False,weights=prpatt.wgt2(X))

Xu.plot.line(x='time')
ua=solver.projectField(Xact,neofs=4,eofscaling=1)
ua.plot.line(x='time')


In [ ]:

outscl=prpatt.adjust_timescales(X,Xact,out.params,ts,f)
outscl

In [ ]:
Xscl=prpatt.imodel_filter_scl(outscl.params,out.params,f)
Xsimscl=prpatt.rmodel(orgeof,Xscl)

In [ ]:
p1=Xsim.weighted(prpatt.wgt(X)).mean(('lat','lon')).plot()
Xact=ds_1pc_anom.tas[0,:150,:,:]

p2=Xact.weighted(prpatt.wgt(X)).mean(('lat','lon')).plot()
p3=Xsimscl.weighted(prpatt.wgt(X)).mean(('lat','lon')).plot()

plt.ylabel(r'(K)')
plt.xlabel('year')
plt.title('1pctCO2 global mean temperature response')
plt.legend((p1[0],p2[0],p3[0]),['Emulated (IR only)','truth','Emulated (IR + quadratic transform)'])

In [ ]:
Xu1=prpatt.imodel_eof(out1.params, f)
Xsim1=prpatt.rmodel(orgeof1,Xu)

In [ ]:
Xsim.weighted(prpatt.wgt(X)).mean(('lat','lon')).plot()
Xact.weighted(prpatt.wgt(X)).mean(('lat','lon')).plot()
Xsim1.weighted(prpatt.wgt(X)).mean(('lat','lon')).plot()


In [ ]:
inma=prpatt.imodel_filter(out.params, f, F0=7.41, y0=1850)

In [ ]:
plt.plot(f+np.square(f)/10)

In [ ]:
inma.plot.line(x='time')

In [ ]:
!git add prpatt.py
!git commit -m 'colab'
!git push origin main